#### Copyright 2023 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Setup

Follow [this guide](https://developers.google.com/zero-touch/guides/customer/service-accounts) to set up your service account.

Download a service account key from the [Google Cloud Console](https://console.cloud.google.com) in JSON format.

Upload this file using the 'Files' tab in the side menu of this colab, and rename the file to 'sa.json'.

### Set up the environment and authenticate with the service

In [ ]:
import json
from apiclient import discovery
import httplib2
from oauth2client.service_account import ServiceAccountCredentials
from google.protobuf import field_mask_pb2

# A single auth scope is used for the zero-touch enrollment customer API.
SCOPES = ['https://www.googleapis.com/auth/androidworkzerotouchemm']
SERVICE_ACCOUNT_KEY_FILE = 'sa.json'

def get_credential():
  """Creates a Credential object with the correct OAuth2 authorization.

  Uses the service account key stored in SERVICE_ACCOUNT_KEY_FILE.

  Returns:
    Credentials, the user's credential.
  """
  credential = ServiceAccountCredentials.from_json_keyfile_name(
    SERVICE_ACCOUNT_KEY_FILE, SCOPES)

  if not credential or credential.invalid:
    print('Unable to authenticate using service account key.')
  return credential


def get_service():
  """Creates a service endpoint for the zero-touch enrollment API.

  Builds and returns an authorized API client service for v1 of the API. Use
  the service endpoint to call the API methods.

  Returns:
    A service Resource object with methods for interacting with the service.
  """
  http_auth = get_credential().authorize(httplib2.Http())
  return discovery.build('androiddeviceprovisioning', 'v1', http=http_auth)

# Create a zero-touch enrollment API service endpoint.
service = get_service()

# Get the customer's account list, first page only.
response = service.customers().list(pageSize=100).execute()

if 'customers' not in response:
  # No accounts found for the user. Confirm the Google Account
  # that authorizes the request can access the zero-touch portal.
  print('No zero-touch enrollment account found.')
else:
  for customer in response['customers']:
    print(customer['name'] + ": " + customer['companyName'])
  CUSTOMER_ACCOUNT = response['customers'][0]['name']

In [ ]:
# Specify a different customer account to use (optional)
CUSTOMER_ACCOUNT = "customers/xxxx" # @param {type:"string"}




---

# DPCs

In [ ]:
# Send an API request to list all the DPCs available using the customer
# account.
dpcResults = service.customers().dpcs().list(parent=CUSTOMER_ACCOUNT).execute()

# Print out the details of each DPC.
for i in range(0,len(dpcResults['dpcs'])):
  # Some DPCs may not have a name, so replace with a marker.
  if 'dpcName' in dpcResults['dpcs'][i]:
    dpcName = dpcResults['dpcs'][i]['dpcName']
  else:
    dpcName = "-"
  print('{0}: name:{1} dpcName:{2}  Package:{3}'.format(i, dpcResults['dpcs'][i]['name'], dpcName, dpcResults['dpcs'][i]['packageName']))

DPC_NAME = dpcResults['dpcs'][0]['name']

In [ ]:
# Specify a different DPC to use (optional)

DPC_NAME = "customers/xxxx/dpcs/yyyy" # @param {type:"string"}



---


# Configurations

## Create a configuration

In [ ]:
CONFIGURATION_NAME = "test"
CONFIGURATION_DPC_NAME = DPC_NAME
CONFIGURATION_COMPANY_NAME = "Test company"
CONFIGURATION_CONTACT_EMAIL = "test@test.com"
CONFIGURATION_CONTACT_PHONE = "5555555555"
CONFIGURATION_IS_DEFAULT = False

configuration = {
  "companyName": CONFIGURATION_COMPANY_NAME,
  "configurationName": CONFIGURATION_NAME,
  "contactEmail": CONFIGURATION_CONTACT_EMAIL,
  "contactPhone": CONFIGURATION_CONTACT_PHONE,
  "dpcResourcePath": CONFIGURATION_DPC_NAME,
  "isDefault": CONFIGURATION_IS_DEFAULT
}

saved_configuration = service.customers().configurations().create(parent=CUSTOMER_ACCOUNT, body=configuration).execute()

print(json.dumps(saved_configuration, indent=4))


## List configurations

In [ ]:
configurations = service.customers().configurations().list(parent=CUSTOMER_ACCOUNT).execute()

print(json.dumps(configurations, indent=4))

## Set a configuration name to use in the following API calls

In [ ]:
# Copy 'name' field from the configuration list above

CONFIGURATION_NAME = "customers/xxxx/configurations/yyyy" # @param {type:"string"}

## Retrieve a named configuration

In [ ]:
configuration = service.customers().configurations().get(name=CONFIGURATION_NAME).execute()

print(json.dumps(configuration, indent=4))

## Delete a named configuration

In [ ]:
service.customers().configurations().delete(name=CONFIGURATION_NAME).execute()

## Patches (updates) a configuration

In [ ]:
# New values to update, or None to keep the same
NEW_CONFIGURATION_NAME = "New test" # @param {type:"raw"}
NEW_CONFIGURATION_DPC_NAME = None # @param {type:"raw"}
NEW_CONFIGURATION_COMPANY_NAME = None # @param {type:"raw"}
NEW_CONFIGURATION_CONTACT_EMAIL = None # @param {type:"raw"}
NEW_CONFIGURATION_CONTACT_PHONE = None # @param {type:"raw"}
NEW_CONFIGURATION_IS_DEFAULT = None # @param {type:"raw"}

configuration = {
}
updateMask = field_mask_pb2.FieldMask()

if (NEW_CONFIGURATION_NAME != None):
  configuration['configurationName'] = NEW_CONFIGURATION_NAME
  updateMask.paths.append('configuration_name')

if (NEW_CONFIGURATION_DPC_NAME != None):
  configuration['dpcResourcePath'] = DPC_NAME
  updateMask.paths.append('dpc_resource_path')

if (NEW_CONFIGURATION_COMPANY_NAME != None):
  configuration['companyName'] = NEW_CONFIGURATION_COMPANY_NAME
  updateMask.paths.append('company_name')

if (NEW_CONFIGURATION_CONTACT_EMAIL != None):
  configuration['contactEmail'] = NEW_CONFIGURATION_CONTACT_EMAIL
  updateMask.paths.append('contact_email')

if (NEW_CONFIGURATION_CONTACT_PHONE != None):
  configuration['contactPhone'] = NEW_CONFIGURATION_CONTACT_PHONE
  updateMask.paths.append('contact_phone')

if (NEW_CONFIGURATION_IS_DEFAULT != None):
  configuration['isDefault'] = NEW_CONFIGURATION_IS_DEFAULT
  updateMask.paths.append('is_default')

print(json.dumps(configuration, indent=4))
print(updateMask.ToJsonString())

saved_configuration = service.customers().configurations().patch(name=CONFIGURATION_NAME, body=configuration, updateMask=updateMask.ToJsonString()).execute()

print(json.dumps(saved_configuration, indent=4))



---


# Devices

## Retrieve list of registered devices

In [ ]:
devices = service.customers().devices().list(parent=CUSTOMER_ACCOUNT, pageSize=100).execute()

print(json.dumps(devices, indent=4))

## Retrieve a named device

In [ ]:
DEVICE_NAME = "customers/xxxx/devices/yyyy" # @param {type:"string"}

device = service.customers().devices().get(name=DEVICE_NAME).execute()

print(json.dumps(device, indent=4))

## Build a DeviceReference using a device id

In [ ]:
# Copy 'name' and 'deviceId' fields from the device list above

DEVICE_ID = "yyyy" # @param {type:"string"}

# Build a DeviceReference using the device id
DEVICE_REFERENCE = {
        'deviceId': DEVICE_ID
    }

## Build a DeviceReference using the device identifiers (WiFi)

In [ ]:
# Replace with real values
DEVICE_MANUFACTURER = 'honeywell' # @param {type:"string"}
DEVICE_MODEL = 'eda51' # @param {type:"string"}
DEVICE_SERIAL = 'TESTTEST1234' # @param {type:"string"}

DEVICE_REFERENCE = {
        'deviceIdentifier': {
            'manufacturer': DEVICE_MANUFACTURER,
            'model': DEVICE_MODEL,
            'serialNumber': DEVICE_SERIAL
        }
    }


## Build a DeviceReference using the device identifiers (cellular)

In [ ]:
# Replace with real value
DEVICE_IMEI = '123456789012345' # @param {type:"string"}

DEVICE_REFERENCE = {
        'deviceIdentifier': {
            'imei': DEVICE_IMEI
        }
    }

## Unclaim a device using the device reference

In [ ]:
body = {
    'device': DEVICE_REFERENCE
}

service.customers().devices().unclaim(parent=CUSTOMER_ACCOUNT, body=body).execute()

## Apply a configuration to a device using the device reference

In [ ]:
body = {
    'device': DEVICE_REFERENCE,
    'configuration': CONFIGURATION_NAME
}

service.customers().devices().applyConfiguration(parent=CUSTOMER_ACCOUNT, body=body).execute()

## Remove a configuration from a device using the device reference

In [ ]:
body = {
    'device': DEVICE_REFERENCE
}

service.customers().devices().removeConfiguration(parent=CUSTOMER_ACCOUNT, body=body).execute()